In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import TensorBoard
import keras
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# code to read desired number of keypoint data instances based on classes or videos. 55 rows per video and 550 rows per class.

start_limit = 11000
end_limit = 16500

X=np.load('/content/drive/MyDrive/60_classes.npy')
X = X[start_limit:end_limit]

Y=np.load('/content/drive/MyDrive/all_labels.npy')
Y = Y[start_limit:end_limit]

In [ ]:
# code to skip over a few augmentations (optional)

new_X = []
new_Y = []

i = 0
while i < X.shape[0]:
    for j in range(15):
      # if j!=2 and j!=3 and j!=6 and j!=7:
      #   new_X.append(X[i+j])
      #   new_Y.append(Y[i+j])
      new_X.append(X[i+j])
      new_Y.append(Y[i+j])
    i += 55

X = np.array(new_X)
Y = np.array(new_Y)

In [ ]:
# splitting and preparing data for training

y = to_categorical(Y).astype(int)
# y = y[:,:]
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
print(X.shape)
print(y.shape)


In [ ]:
del X
del Y

In [ ]:
# model architecture

model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(50,258)))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=False, activation='relu'))

model.add(BatchNormalization())


model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
# setting hyperparameters and training

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)

model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=600, callbacks=[callback],batch_size=64)

In [ ]:
model.save('30C_50F_128B_5T_sorted4.h5')
# del model, X_train, X_test, y_train, y_test

In [ ]:
# accuracy test

yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

0.8444444444444444